In [ ]:
import pandas as pd
import config
l = 0
with open("data/stparkpap/def-dump.txt", encoding="ISO-8859-1") as fileobject:
    definitions = {k: [] for k in config.scrapeTarget["languages"]}
    words_to_read = 5
    for line in fileobject:
        l += 1
        # print(f"Reading line {l}")
        if "--page: " in line: 
            continue
        elif "--word: " in line:
            if words_to_read > 0 and l != 2:
                print(f"The definitions prior to line {l} have < 5 entrees")
            words_to_read = 5
        else:
            # definitions += line.strip()
            if words_to_read == -1:
                continue

            cur_key = config.scrapeTarget["languages"][5 - words_to_read]
            if line.strip() in definitions[cur_key] and cur_key == "pap":
                print(f"{line.strip()} is a pap duplicate entree. Skipping...")
                words_to_read = -1
            else:
                definitions[cur_key].append(line.strip())
                words_to_read -= 1
                
        
df = pd.DataFrame.from_dict(definitions)
print(f"length is {len(df)}")
# df.set_index("pap", inplace=True)
# print(f"length after setting index is {len(df)}")
# print(df)

In [ ]:
import re

def cleanupTokens(definition):
    without_tags = re.sub(r'[\(\[].*?[\)\]]', "", definition)
    return without_tags.strip().lower()

for lan in config.scrapeTarget['languages']:
    simple_col_name = f"{lan}_simple"
    df[simple_col_name] = df.apply(lambda row: cleanupTokens(row[lan]), axis=1)
df["type"] = df.apply(lambda row: "sentence" if len(row.pap.split()) > 1 else "word", axis=1)
df.set_index("pap", inplace=True)

filename = "-".join(config.scrapeTarget["languages"])
df.to_csv(f"data/stparkpap/{filename}.csv")